In [ ]:
import sys
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from bundestag.src.data.ensure_data import get_votes
from bundestag.src.viz.visualize import get_fig_votes

In [ ]:
votes = get_votes()
votes

In [ ]:
vote_map = {
    "yes": "#00aa00",
    "no": "#aa0000",
    "abstain": "#000000",
}

fr = "AfD"
votes_plot = votes.loc[votes.fraction.eq(fr)]



In [ ]:
get_fig_votes(votes_plot)

In [ ]:
df_diss = (
    # look only at dissenting votes:
    votes_plot.loc[~votes_plot.on_party_line]
    # order y-axis by degree of dissent per MdB, select relevant attributes:
    .sort_values("n_dissent", ascending=True)[["name", "label", "party_line", "vote"]]
    .reset_index(drop=True)
)

# 
label_freq = (
    df_diss
    .groupby("label")
    .size()
    .to_frame("freq")
    .reset_index()
    .sort_values(["freq", "label"], ascending=[False, True])
    .reset_index(drop=True)
    .reset_index()[["label", "freq", "index"]]
    .rename({"index": "x"}, axis=1)
)
df_diss = pd.merge(df_diss, label_freq, how="left", on="label")
df_diss.sort_values("x").head(20)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_diss.x,
        y=df_diss.name,
        mode="markers",
        marker=dict(
            size=8,
            symbol="square",
            color="rgba(0,0,128, 1)"
        ),
        customdata=df_diss[["name", "label", "party_line", "vote"]],
        hovertemplate="<b>%{customdata[0]}</b> zur Abstimmung<br>„<i>%{customdata[1]}</i>“<br>Stimme: %{customdata[3]}<br>Fraktionsmehrheit: %{customdata[2]}.<extra></extra>"
    )
)

fig.update_layout(
    width=800,
    height=1500,
    xaxis=dict(
        showticklabels=False
    ),
    margin=dict(t=0, r=0, b=0, l=0)
)
fig.show()